In [ ]:
import sys
sys.path.append("../../")

from src.analysis.cointegration import find_cointegrated_pairs, analyze_pairs, plot_cointegration_heatmap
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

DATE_CONFIG = {
    'TRAIN_START': pd.Timestamp('2021-02-02'),
    'TRAIN_END': pd.Timestamp('2024-01-01'),
    'TEST_END': pd.Timestamp('2025-01-01'),
    'TRADING_DAYS_PER_YEAR': 252  
}

def get_training_period():
    return {
        'start': DATE_CONFIG['TRAIN_START'],
        'end': DATE_CONFIG['TRAIN_END']
    }

def get_test_period():
    return {
        'start': DATE_CONFIG['TRAIN_END'],
        'end': DATE_CONFIG['TEST_END']
    }

def get_training_days():
    years = (DATE_CONFIG['TRAIN_END'] - DATE_CONFIG['TRAIN_START']).days / 365
    return int(years * DATE_CONFIG['TRADING_DAYS_PER_YEAR'])

plt.style.use('classic')
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

In [ ]:
def load_and_prepare_data(file_path):

    df = pd.read_parquet(file_path)
    df['date'] = pd.to_datetime(df['date'])
    
    mask = (df['date'] >= DATE_CONFIG['TRAIN_START']) & \
           (df['date'] <= DATE_CONFIG['TEST_END'])
    df = df[mask]
    
    price_matrix = df.pivot(index='date', columns='symbol', values='close')
    
    symbols = price_matrix.columns.tolist()
    
    print(f"Loaded data from {DATE_CONFIG['TRAIN_START']} to {DATE_CONFIG['TEST_END']}")
    print(f"Total symbols: {len(symbols)}")
    print(f"Total trading days: {len(price_matrix)}")
    
    return price_matrix, symbols

In [ ]:
def generate_pairs(cointegrated_pairs):
    return cointegrated_pairs

In [ ]:
import toml

with open("../config.toml", "r") as f:
    config = toml.load(f)
    
price_matrix, symbols = load_and_prepare_data(config['data']['raw_data_path'])

In [ ]:
def calculate_returns_and_spreads(price_matrix, cointegrated_pairs):
    returns = price_matrix.pct_change().dropna()
    
    pairs = generate_pairs(cointegrated_pairs)
    
    spreads = pd.DataFrame(index=returns.index)
    for s1, s2 in pairs:
        spreads[f'{s1}_{s2}_spread'] = returns[s1] - returns[s2]
        
    return returns, spreads

In [ ]:
def prepare_ml_data(returns, spreads, train_period, test_period, lookback=3):
    ml_datasets = {}
    
    for spread_col in spreads.columns:
        sym1, sym2 = spread_col.replace('_spread', '').split('_')
        
        df = pd.DataFrame({
            f'{sym1}_return': returns[sym1],
            f'{sym2}_return': returns[sym2]
        })
        
        for t in range(1, lookback+1):
            df[f'{sym1}_return_t-{t}'] = df[f'{sym1}_return'].shift(t)
            df[f'{sym2}_return_t-{t}'] = df[f'{sym2}_return'].shift(t)
        
        spread_next_day = (df[f'{sym1}_return'] - df[f'{sym2}_return']).shift(-1)
        df['target'] = np.where(spread_next_day > 0, 1, 0)
        
        feature_cols = [col for col in df.columns if 't-' in col]
        features = df[feature_cols].copy()
        
        clean_idx = features.dropna().index
        features = features.loc[clean_idx]
        target = df.loc[clean_idx, 'target']
    
        train_mask = (features.index >= train_period['start']) & (features.index < train_period['end'])
        test_mask = (features.index >= test_period['start']) & (features.index < test_period['end'])
        
        ml_datasets[f'{sym1}_{sym2}'] = {
            'X_train': features[train_mask],
            'X_test': features[test_mask],
            'y_train': target[train_mask],
            'y_test': target[test_mask]
        }
        
    return ml_datasets

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def train_evaluate_models(ml_datasets, coint_results):
   results = {}
   
   for pair, data in tqdm(ml_datasets.items(), desc="Training models"):
       sym1, sym2 = pair.split('_')
       
       # Debug: Klassen-Balance
       print(f"\nAnalyzing pair: {pair}")
       print("Class distribution (train):", np.bincount(data['y_train']) / len(data['y_train']))
       print("Class distribution (test):", np.bincount(data['y_test']) / len(data['y_test']))
       
       p_value = coint_results[
           ((coint_results['symbol1'] == sym1) & (coint_results['symbol2'] == sym2)) |
           ((coint_results['symbol1'] == sym2) & (coint_results['symbol2'] == sym1))
       ]['p_value'].iloc[0]
       
       scaler = StandardScaler()
       X_train_scaled = scaler.fit_transform(data['X_train'])
       X_test_scaled = scaler.transform(data['X_test'])
       
       # Angepasste Parameter
       svm = SVC(
           kernel='rbf',
           C=0.1,  # Reduzierter C-Wert
           class_weight='balanced',
           random_state=42
       )
       svm.fit(X_train_scaled, data['y_train'])
       predictions = svm.predict(X_test_scaled)
       
       # Debug: Predictions Verteilung
       print("\nPrediction distribution:", np.bincount(predictions) / len(predictions))
       
       f1 = f1_score(data['y_test'], predictions)
       weighted_score = 1 * f1 + 0 * (1 - p_value)
       
       results[pair] = {
           'model': svm,
           'scaler': scaler,
           'accuracy': accuracy_score(data['y_test'], predictions),
           'precision': precision_score(data['y_test'], predictions),
           'recall': recall_score(data['y_test'], predictions),
           'f1': f1,
           'p_value': p_value,
           'weighted_score': weighted_score
       }
   
   return results

In [ ]:
score_matrix, pvalue_matrix, cointegrated_pairs, coint_results = analyze_pairs(price_matrix)

returns, spreads = calculate_returns_and_spreads(price_matrix, cointegrated_pairs)

train_period = get_training_period()
test_period = get_test_period()

ml_datasets = prepare_ml_data(returns, spreads, train_period, test_period)

model_results = train_evaluate_models(ml_datasets, coint_results)

In [40]:
results_df = pd.DataFrame([
   {
       'pair': pair,
       'accuracy': metrics['accuracy'],
       'precision': metrics['precision'],
       'recall': metrics['recall'],
       'f1': metrics['f1'],
       'p_value': metrics['p_value'],
       'weighted_score': metrics['weighted_score']
   }
   for pair, metrics in model_results.items()
])

top_20 = results_df.sort_values('weighted_score', ascending=False).head(20)
print(top_20[['pair', 'f1', 'p_value', 'weighted_score', 'accuracy', 'precision', 'recall']])

          pair        f1   p_value  weighted_score  accuracy  precision  \
22     AMAT_MU  0.697051  0.039682        0.697051  0.551587   0.555556   
220   NFLX_TTD  0.684211  0.030740        0.684211  0.523810   0.528455   
14     ADP_KHC  0.664820  0.013673        0.664820  0.519841   0.519481   
167   KDP_LRCX  0.661017  0.039539        0.661017  0.523810   0.522321   
176   KDP_SNPS  0.660920  0.024321        0.660920  0.531746   0.529954   
223   NXPI_XEL  0.659218  0.048490        0.659218  0.515873   0.531532   
165   ISRG_TTD  0.657143  0.003670        0.657143  0.523810   0.511111   
199  LRCX_NXPI  0.656977  0.001418        0.656977  0.531746   0.520737   
196   LIN_PANW  0.656000  0.027741        0.656000  0.488095   0.488095   
65    CSX_DXCM  0.655914  0.043234        0.655914  0.492063   0.489960   
11    ADI_PANW  0.652291  0.009833        0.652291  0.488095   0.485944   
80    CSX_LRCX  0.648352  0.014721        0.648352  0.492063   0.489627   
112   EA_GOOGL  0.647230 